In [ ]:
!pip install transformers==4.28.0 -q
!pip install datasets -q

In [ ]:
from google.colab import files, drive
import pandas as pd
import re
from tqdm.auto import tqdm
import json

drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import nltk
from sklearn.metrics.pairwise import cosine_similarity

nltk.download('punkt') 

In [ ]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('bert-base-nli-mean-tokens')

# Metrics and data downloading

In [ ]:
from datasets import load_dataset

dataset = load_dataset('ra4wv2/qa')

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/ra4wv2___csv/ra4wv2--qa-703fa30e467874d5/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
unnes_asps = ['better', 'difference', 'worse', 'different', 'win', 'best', 
              'worst', 'differences', 'bad', 'good', 'common', 'same', 
              'alike', 'like', 'likes', 'do', '-', 'than', 'as']

In [ ]:
def delete_unnes_aspects(df, unnes_asps):
  df['aspect'].fillna('', inplace=True)
  for asp in unnes_asps:
    df['aspect'].replace(asp, '', inplace=True)
  return df

In [ ]:
def cosine_sim(text1, text2):
    emb1 = [model.encode(text1)]
    emb2 = [model.encode(text2)]
    return cosine_similarity(emb1, emb2)[0][0]

In [ ]:
def evaluate_muc_ceaf_bcubed_f1(df, answer_aspect, answer_objects):
  muc_objects, muc_aspect = [], []
  ceaf_objects, ceaf_aspect = [], []

  for i in range(len(df)):
    objs = [df.iloc[i,1].lower(), df.iloc[i,2].lower()]
    objs = [obj+'_' if len(obj)==1 else obj for obj in objs]
    aspect = df.iloc[i,3].lower()

    pred_objects = [ans.lower() for ans in answer_objects[i]]
    pred_objects = [obj+'_' if len(obj)==1 else obj for obj in pred_objects]
    pred_aspect = answer_aspect[i].lower()

    if aspect == '':
      split_aspect = [['_', '_']]
    elif len(aspect) == 1:
      split_aspect = [[i for i in aspect]]
      split_aspect[0].append('_')
    else:
      split_aspect = [[i for i in aspect]]

    if pred_aspect == '':
      split_pred_aspect = [['_', '_']]
    elif len(pred_aspect) == 1:
      split_pred_aspect = [[i for i in pred_aspect]]
      split_pred_aspect[0].append('_')
    else:
      split_pred_aspect = [[i for i in pred_aspect]]

    split_objs = [[i for i in obj] for obj in objs]
    if pred_objects == []:
      split_pred_objs = [['_', '_'], ['_', '_']]
    elif len(pred_objects) == 1:
      split_pred_objs = [[i for i in obj] for obj in pred_objects]
      split_pred_objs.append(['_', '_'])
    else:
      split_pred_objs = [[i for i in obj] for obj in pred_objects]

    muc_objects.append(muc(split_pred_objs, split_objs))
    muc_aspect.append(muc(split_pred_aspect, split_aspect))

    ceaf_objects.append(ceaf(split_pred_objs, split_objs))
    ceaf_aspect.append(ceaf(split_pred_aspect, split_aspect))


  print('MUC')
  print('Objects:')
  print(f'''  precision: {sum([m[0] for m in muc_objects])/len(muc_objects)}
  recall: {sum([m[1] for m in muc_objects])/len(muc_objects)}
  f-score: {sum([m[2] for m in muc_objects])/len(muc_objects)}''')
  print('Aspect:')
  print(f'''  precision: {sum([m[0] for m in muc_aspect])/len(muc_aspect)}
  recall: {sum([m[1] for m in muc_aspect])/len(muc_aspect)}
  f-score: {sum([m[2] for m in muc_aspect])/len(muc_aspect)}''')
  
  print('\nCEAF')
  print('Objects:')
  print(f'''  precision: {sum([c[0] for c in ceaf_objects])/len(ceaf_objects)}
  recall: {sum([c[1] for c in ceaf_objects])/len(ceaf_objects)}
  f-score: {sum([c[2] for c in ceaf_objects])/len(ceaf_objects)}''')
  print('Aspect:')
  print(f'''  precision: {sum([c[0] for c in ceaf_aspect])/len(ceaf_aspect)}
  recall: {sum([c[1] for c in ceaf_aspect])/len(ceaf_aspect)}
  f-score: {sum([c[2] for c in ceaf_aspect])/len(ceaf_aspect)}''')
  



In [ ]:
def evaluate_prec_rec_f1(df, answer_aspect, answer_objects, ed):
  tp_objects = 0
  fp_objects = 0
  fn_objects = 0

  tp_aspect = 0
  fp_aspect = 0
  fn_aspect = 0  

  for i in range(len(df)):
    objs = [df.iloc[i,1].lower(), df.iloc[i,2].lower()]
    aspect = df.iloc[i,3].lower()

    pred_aspect = answer_aspect[i].lower()
    pred_objects = [ans.lower() for ans in answer_objects[i]][:2]

    objs = [re.sub('[^a-z0-9]+the ', ' ', obj) for obj in objs]
    objs = [re.sub(' +', ' ', obj) for obj in objs]

    if set(objs) == set(pred_objects):
      tp_objects += 2
    else:
      if len(pred_objects) == 2:
        if nltk.edit_distance(objs[0], pred_objects[0], transpositions=False)<=ed: 
          tp_objects += 1
        else:
          fp_objects += 1
        if nltk.edit_distance(objs[1], pred_objects[1], transpositions=False)<=ed: 
          tp_objects += 1
        else:
          fp_objects += 1
      elif not pred_objects:
        fn_objects += 2
      elif len(pred_objects) == 1:
        max_added = 0 
        for obj in objs: 
          if nltk.edit_distance(obj, pred_objects[0], transpositions=False)<=ed: 
            max_added += 1 
        if max_added > 0: 
          tp_objects += 1 
        else: 
          fp_objects += 1 

    if nltk.edit_distance(aspect, pred_aspect, transpositions=False)<=ed: 
        tp_aspect += 1
    else:
      if pred_aspect:
        fp_aspect += 1
      else:
        fn_aspect += 1
  
  precision = tp_objects/(tp_objects+fp_objects) 
  recall = tp_objects/(tp_objects+fn_objects) 
  f1 = (2*precision*recall)/(precision+recall)
  print('Objects:')
  print(f'precision: {precision}\nrecall: {recall}\nf1: {f1}')

  precision = tp_aspect/(tp_aspect+fp_aspect) 
  recall = tp_aspect/(tp_aspect+fn_aspect) 
  f1 = (2*precision*recall)/(precision+recall)
  print('Aspects:')
  print(f'precision: {precision}\nrecall: {recall}\nf1: {f1}')

# Finetuned T5 evaluation

## generating

In [ ]:
from transformers import AutoTokenizer, T5ForConditionalGeneration

tokenizer = AutoTokenizer.from_pretrained("ra4wv2/t5-large-qa")
model = T5ForConditionalGeneration.from_pretrained("ra4wv2/t5-large-qa")

In [ ]:
def generator(inp_text):
  input_ids = tokenizer(inp_text, return_tensors="pt").input_ids

  outputs = model.generate(input_ids, max_length=100)
  preds = [
    tokenizer.decode(out, skip_special_tokens=True)
    for out in outputs
    ]
  return preds[0]

In [ ]:
answers = []
for i in tqdm(range(len(dataset["test"]))):
  answer = generator(dataset["test"][i]['instruction'])
  answers.append(answer)

  0%|          | 0/612 [00:00<?, ?it/s]

In [ ]:
ids = [item['id'] for item in dataset['test']]
test_df = df.iloc[ids]
test_df['t5_answers'] = answers

In [ ]:
test_df.to_csv('./drive/My Drive/finetuned_t5_zero.csv', index=False, encoding='utf-8', sep='\t')

## evaluation

In [ ]:
test_df = pd.read_csv('./drive/My Drive/finetuned_t5_zero.csv', encoding='utf-8', sep='\t')

In [ ]:
test_df = delete_unnes_aspects(test_df, unnes_asps)

In [ ]:
answer_objects = []
answer_aspect = []

for i in range(len(test_df)):
  answer = test_df.iloc[i,4]
  if 'Aspect' in answer:
    aspect = answer.split('Aspect')[1].strip(' ').strip(':')
    objj = answer.split('Aspect')[0]
    objj = re.sub('Objects', '', objj).strip(' ').strip(':')
  else:
    aspect = ''
    if 'Objects' in answer:
      objj = re.sub('Objects', '', answer).strip(' ').strip(':')
    else:
      objj = ''
  if "'" in objj:
    obj = re.sub("' ?, '", "','", objj)
    obj = re.sub("'", '', obj).strip(' ').strip('\'').split(",")
  else:
    obj = re.sub(' ?, ', ',', objj).strip(' ').strip('\'').split(',')

  answer_objects.append(obj)
  answer_aspect.append(aspect.strip(' ').strip('\''))

In [ ]:
clear_answer_aspect = []
for a in answer_aspect:
  if a in unnes_asps:
    clear_answer_aspect.append('')
  else:
    clear_answer_aspect.append(a)   

answer_aspect = clear_answer_aspect 

In [ ]:
evaluate_prec_rec_f1(test_df, answer_aspect, answer_objects, 3) 

Objects:
precision: 0.7238562091503268
recall: 1.0
f1: 0.8398104265402843
Aspects:
precision: 0.7592267135325131
recall: 0.9094736842105263
f1: 0.8275862068965517


### MUC, CEAF

In [ ]:
!pip install metric4coref -q

  Preparing metadata (setup.py) ... done


In [ ]:
from metric4coref import muc, ceaf

In [ ]:
evaluate_muc_ceaf_bcubed_f1(test_df, answer_aspect, answer_objects)

MUC
Objects:
  precision: 0.9144709651979621
  recall: 0.9221853283751353
  f-score: 0.888382510832487
Aspect:
  precision: 0.7763894575662019
  recall: 0.7731767976238612
  f-score: 0.7485888750635039

CEAF
Objects:
  precision: 0.7075619730948739
  recall: 0.7164149811644189
  f-score: 0.6942959515415112
Aspect:
  precision: 0.5294657726754899
  recall: 0.5277798234352784
  f-score: 0.5159720119074924


### edit-distance

In [ ]:
object_distance = []
aspect_distance = []

for i in range(len(test_df)):
  objs = [test_df.iloc[i,1].lower(), test_df.iloc[i,2].lower()]
  aspect = test_df.iloc[i,3].lower()

  pred_aspect = answer_aspect[i].lower()    
  pred_objects = [ans.lower() for ans in answer_objects[i]]

  if set(objs) == set(pred_objects):
    object_distance.append(0)
  elif len(objs) == len(pred_objects):
    ed_obj1 = nltk.edit_distance(objs[0], pred_objects[0], transpositions=False)
    ed_obj2 = nltk.edit_distance(objs[1], pred_objects[1], transpositions=False)
    object_distance.append((ed_obj1+ed_obj2)/2)
  # else: # добавить, если считаем и те, которые не нашлись
  #   object_distance.append((len(objs[0])+len(objs[1]))/2)

  if pred_aspect: # убрать, если считаем и те, которые не найдены 
    aspect_distance.append(nltk.edit_distance(pred_aspect, aspect, transpositions=False))
  if not pred_aspect and not aspect:
    aspect_distance.append(0)


print(sum(object_distance)/len(object_distance))
print(sum(aspect_distance)/len(aspect_distance))

3.290983606557377
2.325088339222615


### strict

full match

In [ ]:
correct_answers = 0
correct_aspects = 0

for i in range(len(test_df)):
  objs = [test_df.iloc[i,1].lower(), test_df.iloc[i,2].lower()]
  aspect = test_df.iloc[i,3].lower()

  pred_aspect = answer_aspect[i].lower()
  pred_objects = [ans.lower() for ans in answer_objects[i]]

  objs = [re.sub('[^a-z0-9]+the ', ' ', obj) for obj in objs]
  objs = [re.sub(' +', ' ', obj) for obj in objs]

  if set(objs) == set(pred_objects):
    correct_answers += 1

  if pred_aspect == aspect:
    correct_aspects += 1

print(correct_answers/len(test_df))
print(correct_aspects/len(test_df))

0.6062091503267973
0.6911764705882353


cosine similarity

In [ ]:
cos_objects = []
cos_aspects = []

for i in tqdm(range(len(test_df))):
  objs = [test_df.iloc[i,1].lower(), test_df.iloc[i,2].lower()]
  aspect = test_df.iloc[i,3].lower()

  pred_aspect = answer_aspect[i].lower()
  pred_objs = [ans.lower() for ans in answer_objects[i]]

  if len(objs) == len(pred_objs):
    ob1_cos = cosine_sim(pred_objs[0], objs[0])
    ob2_cos = cosine_sim(pred_objs[1], objs[1])
    cos_objects.append((ob1_cos+ob2_cos)/2)
  else:
    cos_objects.append(0)

  if aspect and pred_aspect:
    cos_aspects.append(cosine_sim(pred_aspect, aspect))
  elif not aspect and not pred_aspect:
    cos_aspects.append(1)
  else:
    cos_aspects.append(0)

print(sum(cos_objects)/len(cos_objects))
print(sum(cos_aspects)/len(cos_aspects))

  0%|          | 0/612 [00:00<?, ?it/s]

0.9417118683846948
0.8079041434464111


cosine similarity for found only elements

In [ ]:
cos_objects = []
cos_aspects = []

for i in tqdm(range(len(test_df))):
  objs = [test_df.iloc[i,1].lower(), test_df.iloc[i,2].lower()]
  aspect = test_df.iloc[i,3].lower()

  pred_aspect = answer_aspect[i].lower()
  pred_objs = [ans.lower() for ans in answer_objects[i]]

  if len(objs) == len(pred_objs):
    ob1_cos = cosine_sim(pred_objs[0], objs[0])
    ob2_cos = cosine_sim(pred_objs[1], objs[1])
    cos_objects.append((ob1_cos+ob2_cos)/2) 
  if len(pred_objs) == 1:
    ob1_cos = cosine_sim(pred_objs[0], objs[0])
    ob2_cos = cosine_sim(pred_objs[0], objs[1])
    cos_objects.append(max(ob1_cos, ob2_cos))

  if aspect and pred_aspect:
    cos_aspects.append(cosine_sim(pred_aspect, aspect))
  elif not aspect and not pred_aspect:
    cos_aspects.append(1)

print(sum(cos_objects)/len(cos_objects))
print(sum(cos_aspects)/len(cos_aspects))

  0%|          | 0/612 [00:00<?, ?it/s]

0.944799448281038
0.9490159995954005


### relaxed

full match

In [ ]:
correct_answers = 0
correct_aspects = 0

for i in range(len(test_df)):
  objs = [test_df.iloc[i,1].lower(), test_df.iloc[i,2].lower()]
  aspect = test_df.iloc[i,3].lower()

  pred_aspect = answer_aspect[i].lower()
  pred_objects = [ans.lower() for ans in answer_objects[i]]

  objs = [re.sub('[^a-z0-9]+the ', ' ', obj) for obj in objs]
  objs = [re.sub(' +', ' ', obj) for obj in objs]
  
  for obj in pred_objects:
    if obj in objs:
      correct_answers += 0.5

  # else:
  #   print(objs, pred_objects)
  #   print('###')

  if pred_aspect == aspect:
    correct_aspects += 1

print(correct_answers/len(test_df))
print(correct_aspects/len(test_df))

0.6781045751633987
0.6911764705882353


cosine similarity

In [ ]:
cos_objects = []
cos_aspects = []

for i in tqdm(range(len(test_df))):
  objs = [test_df.iloc[i,1].lower(), test_df.iloc[i,2].lower()]
  aspect = test_df.iloc[i,3].lower()

  pred_aspect = answer_aspect[i].lower()
  pred_objs = [ans.lower() for ans in answer_objects[i]]

  if len(pred_objs) == 1:
    max_cos = 0
    for obj in objs:
      obj_cos = cosine_sim(pred_objs[0], obj)
      if obj_cos > max_cos:
        max_cos = obj_cos
    cos_objects.append(max_cos*0.5)

  elif len(objs) == len(pred_objs):
    ob1_cos = cosine_sim(pred_objs[0], objs[0])
    ob2_cos = cosine_sim(pred_objs[1], objs[1])
    cos_objects.append((ob1_cos+ob2_cos)/2)

  else:
    cos_objects.append(0)

  if aspect and pred_aspect:
    cos_aspects.append(cosine_sim(pred_aspect, aspect))
  elif not aspect and not pred_aspect:
    cos_aspects.append(1)
  else:
    cos_aspects.append(0)

print(sum(cos_objects)/len(cos_objects))
print(sum(cos_aspects)/len(cos_aspects))

  0%|          | 0/612 [00:00<?, ?it/s]

0.9417118683846948
0.8079041434464111


# Finetuned Flan-T5 evaluation

## generating

In [ ]:
from transformers import AutoTokenizer, T5ForConditionalGeneration

tokenizer = AutoTokenizer.from_pretrained("ra4wv2/flan-t5-large-qa")
model = T5ForConditionalGeneration.from_pretrained("ra4wv2/flan-t5-large-qa")

In [ ]:
def generator(inp_text):
  input_ids = tokenizer(inp_text, return_tensors="pt").input_ids

  outputs = model.generate(input_ids, max_length=100)
  preds = [
    tokenizer.decode(out, skip_special_tokens=True)
    for out in outputs
    ]
  return preds[0]

In [ ]:
answers = []
for i in tqdm(range(len(dataset["test"]))): 
  answer = generator(dataset["test"][i]['instruction'])
  answers.append(answer)

  0%|          | 0/612 [00:00<?, ?it/s]

In [ ]:
ids = [item['id'] for item in dataset['test']]
test_df = df.iloc[ids]
test_df['flant5_answers'] = answers

<ipython-input-10-10ddd7295574>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['flant5_answers'] = answers


In [ ]:
test_df.to_csv('./drive/My Drive/finetuned_flant5_zero.csv', index=False, encoding='utf-8', sep='\t')

## evaluating

In [ ]:
test_df = pd.read_csv('./drive/My Drive/finetuned_flant5_zero.csv', encoding='utf-8', sep='\t')

In [ ]:
test_df = delete_unnes_aspects(test_df, unnes_asps)

In [ ]:
answer_objects = []
answer_aspect = []

for i in range(len(test_df)):
  answer = test_df.iloc[i,4]
  if 'Aspect' in answer:
    aspect = answer.split('Aspect')[1].strip(' ').strip(':')
    objj = answer.split('Aspect')[0]
    objj = re.sub('Objects', '', objj).strip(' ').strip(':')
  else:
    aspect = ''
    if 'Objects' in answer:
      objj = re.sub('Objects', '', answer).strip(' ').strip(':')
    else:
      objj = ''
  if "'" in objj:
    obj = re.sub("' ?, '", "','", objj)
    obj = re.sub("'", '', obj).strip(' ').strip('\'').split(",")
  else:
    obj = re.sub(' ?, ', ',', objj).strip(' ').strip('\'').split(',')

  answer_objects.append(obj)
  answer_aspect.append(aspect.strip(' ').strip('\''))

In [ ]:
clear_answer_aspect = []
for a in answer_aspect:
  if a in unnes_asps:
    clear_answer_aspect.append('')
  else:
    clear_answer_aspect.append(a)   

answer_aspect = clear_answer_aspect 

In [ ]:
evaluate_prec_rec_f1(test_df, answer_aspect, answer_objects, 3) 

Objects:
precision: 0.7377450980392157
recall: 1.0
f1: 0.849083215796897
Aspects:
precision: 0.7712765957446809
recall: 0.9006211180124224
f1: 0.8309455587392551


### MUC, CEAF

In [ ]:
evaluate_muc_ceaf_bcubed_f1(test_df, answer_aspect, answer_objects)

MUC
Objects:
  precision: 0.9125434396956986
  recall: 0.9316779844071365
  f-score: 0.8951909507184415
Aspect:
  precision: 0.7763250442882506
  recall: 0.7798039511849757
  f-score: 0.7527425551773139

CEAF
Objects:
  precision: 0.7061171078845422
  recall: 0.7194889717936103
  f-score: 0.697019492645627
Aspect:
  precision: 0.5276348568868472
  recall: 0.5297345897515437
  f-score: 0.5173253809886629


### edit-distance

In [ ]:
object_distance = []
aspect_distance = []

for i in range(len(test_df)):
  objs = [test_df.iloc[i,1].lower(), test_df.iloc[i,2].lower()]
  aspect = test_df.iloc[i,3].lower()

  pred_aspect = answer_aspect[i].lower()    
  pred_objects = [ans.lower() for ans in answer_objects[i]]

  if set(objs) == set(pred_objects):
    object_distance.append(0)
  elif len(objs) == len(pred_objects):
    ed_obj1 = nltk.edit_distance(objs[0], pred_objects[0], transpositions=False)
    ed_obj2 = nltk.edit_distance(objs[1], pred_objects[1], transpositions=False)
    object_distance.append((ed_obj1+ed_obj2)/2)

  if pred_aspect: 
    aspect_distance.append(nltk.edit_distance(pred_aspect, aspect, transpositions=False))
  if not pred_aspect and not aspect:
    aspect_distance.append(0)


print(sum(object_distance)/len(object_distance))
print(sum(aspect_distance)/len(aspect_distance))

3.3513957307060753
2.0551601423487544


### strict

full match

In [ ]:
correct_answers = 0
correct_aspects = 0

for i in range(len(test_df)):
  objs = [test_df.iloc[i,1].lower(), test_df.iloc[i,2].lower()]
  aspect = test_df.iloc[i,3].lower()

  pred_aspect = answer_aspect[i].lower()
  pred_objects = [ans.lower() for ans in answer_objects[i]]

  objs = [re.sub('[^a-z0-9]+the ', ' ', obj) for obj in objs]
  objs = [re.sub(' +', ' ', obj) for obj in objs]

  if set(objs) == set(pred_objects):
    correct_answers += 1

  if pred_aspect == aspect:
    correct_aspects += 1

print(correct_answers/len(test_df))
print(correct_aspects/len(test_df))

0.6094771241830066
0.6879084967320261


cosine similarity

In [ ]:
cos_objects = []
cos_aspects = []

for i in tqdm(range(len(test_df))):
  objs = [test_df.iloc[i,1].lower(), test_df.iloc[i,2].lower()]
  aspect = test_df.iloc[i,3].lower()

  pred_aspect = answer_aspect[i].lower()
  pred_objs = [ans.lower() for ans in answer_objects[i]]

  if len(objs) == len(pred_objs):
    ob1_cos = cosine_sim(pred_objs[0], objs[0])
    ob2_cos = cosine_sim(pred_objs[1], objs[1])
    cos_objects.append((ob1_cos+ob2_cos)/2)
  else:
    cos_objects.append(0)

  if aspect and pred_aspect:
    cos_aspects.append(cosine_sim(pred_aspect, aspect))
  elif not aspect and not pred_aspect:
    cos_aspects.append(1)
  else:
    cos_aspects.append(0)

print(sum(cos_objects)/len(cos_objects))
print(sum(cos_aspects)/len(cos_aspects))

  0%|          | 0/612 [00:00<?, ?it/s]

0.9421787019453797
0.8076313729184905


cosine similarity for found only elements

In [ ]:
cos_objects = []
cos_aspects = []

for i in tqdm(range(len(test_df))):
  objs = [test_df.iloc[i,1].lower(), test_df.iloc[i,2].lower()]
  aspect = test_df.iloc[i,3].lower()

  pred_aspect = answer_aspect[i].lower()
  pred_objs = [ans.lower() for ans in answer_objects[i]]

  if len(objs) == len(pred_objs):
    ob1_cos = cosine_sim(pred_objs[0], objs[0])
    ob2_cos = cosine_sim(pred_objs[1], objs[1])
    cos_objects.append((ob1_cos+ob2_cos)/2) 
  if len(pred_objs) == 1:
    ob1_cos = cosine_sim(pred_objs[0], objs[0])
    ob2_cos = cosine_sim(pred_objs[0], objs[1])
    cos_objects.append(max(ob1_cos, ob2_cos))

  if aspect and pred_aspect:
    cos_aspects.append(cosine_sim(pred_aspect, aspect))
  elif not aspect and not pred_aspect:
    cos_aspects.append(1)

print(sum(cos_objects)/len(cos_objects))
print(sum(cos_aspects)/len(cos_aspects))

  0%|          | 0/612 [00:00<?, ?it/s]

0.9468199763391992
0.9486955858466721


### relaxed

full match

In [ ]:
correct_answers = 0
correct_aspects = 0

for i in range(len(test_df)):
  objs = [test_df.iloc[i,1].lower(), test_df.iloc[i,2].lower()]
  aspect = test_df.iloc[i,3].lower()

  pred_aspect = answer_aspect[i].lower()
  pred_objects = [ans.lower() for ans in answer_objects[i]]

  objs = [re.sub('[^a-z0-9]+the ', ' ', obj) for obj in objs]
  objs = [re.sub(' +', ' ', obj) for obj in objs]
  
  for obj in pred_objects:
    if obj in objs:
      correct_answers += 0.5

  if pred_aspect == aspect:
    correct_aspects += 1

print(correct_answers/len(test_df))
print(correct_aspects/len(test_df))

0.6887254901960784
0.6879084967320261


cosine similarity

In [ ]:
cos_objects = []
cos_aspects = []

for i in tqdm(range(len(test_df))):
  objs = [test_df.iloc[i,1].lower(), test_df.iloc[i,2].lower()]
  aspect = test_df.iloc[i,3].lower()

  pred_aspect = answer_aspect[i].lower()
  pred_objs = [ans.lower() for ans in answer_objects[i]]

  if len(pred_objs) == 1:
    max_cos = 0
    for obj in objs:
      obj_cos = cosine_sim(pred_objs[0], obj)
      if obj_cos > max_cos:
        max_cos = obj_cos
    cos_objects.append(max_cos*0.5)

  elif len(objs) == len(pred_objs):
    ob1_cos = cosine_sim(pred_objs[0], objs[0])
    ob2_cos = cosine_sim(pred_objs[1], objs[1])
    cos_objects.append((ob1_cos+ob2_cos)/2)

  else:
    cos_objects.append(0)

  if aspect and pred_aspect:
    cos_aspects.append(cosine_sim(pred_aspect, aspect))
  elif not aspect and not pred_aspect:
    cos_aspects.append(1)
  else:
    cos_aspects.append(0)

print(sum(cos_objects)/len(cos_objects))
print(sum(cos_aspects)/len(cos_aspects))

  0%|          | 0/612 [00:00<?, ?it/s]

0.9421787019453797
0.8076313729184905
